In [1]:
!pip3 install transformers
!python3 -m pip install tensorflow

from google.colab import drive
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from transformers import TFBertModel, BertTokenizer

In [3]:
#Préparation des données:

## Charger les jeux de données
train_df = pd.read_csv('/content/drive//MyDrive/goodreads_train.csv')
test_df = pd.read_csv('/content/drive//MyDrive/goodreads_test.csv')

print(len(train_df))
print(len(test_df))

900000
478033


In [4]:
# Instanciation du tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# Fonction pour convertir les textes en entrée en représentations numériques
def encode_text(texts, tokenizer, max_len=512):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='np'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)

In [64]:
# Conversion des textes d'entraînement et de test en entrées numériques
train_input_ids, train_attention_masks = encode_text(train_df['review_text'].head(100000), tokenizer)
test_input_ids, test_attention_masks = encode_text(test_df['review_text'], tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: ignored

In [67]:
def build_model():
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    input_ids = Input(shape=(512,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(512,), dtype=tf.int32, name='attention_mask')
    bert_output = bert_model({'input_ids': input_ids, 'attention_mask': attention_mask})
    output = Dense(6, activation='softmax')(bert_output[1])
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(loss='mse', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
    return model

In [68]:
train_input_ids = train_input_ids.reshape(train_input_ids.shape[0], -1)
train_attention_masks = train_attention_masks.reshape(train_attention_masks.shape[0], -1)
test_input_ids = test_input_ids.reshape(test_input_ids.shape[0], -1)
test_attention_masks = test_attention_masks.reshape(test_attention_masks.shape[0], -1)

In [ ]:
# Entraînement du modèle
model = build_model()
model.fit(
    [train_input_ids, train_attention_masks],
    train_df['rating'].values,
    validation_split=0.2,
    epochs=5,
    batch_size=8
)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/5
 5074/10000 [==============>...............] - ETA: 1:17:53 - loss: 14.1895 - accuracy: 0.1923

In [ ]:
# Prédiction sur les données de test
predictions = model.predict([test_input_ids, test_attention_masks]).flatten()

In [ ]:
## Enregistrement des prédictions dans un fichier csv
test_df['rating'] = test_pred_labels
test_df[['review_id', 'rating']].to_csv('/content/drive/MyDrive/transformers_submission.csv', index=False)